# Boltzmann Machines
https://www.cs.toronto.edu/~hinton/csc321/readings/boltz321.pdf  
https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf  
http://yann.lecun.com/exdb/publis/pdf/lecun-06.pdf  
https://en.wikipedia.org/wiki/Boltzmann_distribution  
https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15  
https://skymind.ai/wiki/restricted-boltzmann-machine  